## Classifier and preprocessing

In this notebook, the noironicos dataset will be treated, since ironicos's tweets are all ironic and we want a mixture of ironic and non ironic.

In [48]:
# General import and load data
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
import re

# Import database
df_noironicos=pd.read_csv('noironicos_bodies.csv')

# Encode categorical variable (ironic)
df_noironicos.loc[df_noironicos['ironic']==True, "ironic"] = 1
df_noironicos.loc[df_noironicos['ironic']==False, "ironic"] = 0

# Drop non-used columns
df_noironicos.drop(['id_tweet', 'depends_image', 'depends_link', 'depends_retweet'], axis=1, inplace=True)

# Drop nan rows
df_clean=df_noironicos.dropna(subset=['text'])
df_noironicos=df_clean

# Final dataset
df_noironicos.head()

,ironic,text
0,1,Algunas personas sufren en las discos mientras...
3,1,@jacevedoaraya es para sostener el marcador......
4,1,Alguna de estas imágenes te sacara una sonrisa...
5,1,@_Eurovision2014 en 2013 falta esdm jajajajaja...
6,1,Hooo que buen padre...#Sarcasmo #GH2015


In [49]:
# Define X and Y
X = df_noironicos['text'].values
y = df_noironicos['ironic'].values

## Lexical features
The lexical features analysis will be performed by using the twitter tokenizer provided by nltk library.

PUEDE QUE HAYA UN PROBLEMA EN LA PARTE DE LOS EMOTICONOS

In [54]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string

# Insert all words of the tweets in a list
tokens_ar = []
lemmas_ar = []
# Used later for cleansing
ht = re.compile(r'http.')
bar = re.compile(r'//*')
pr = ["rt","@","http","https","'s",'...', 'english', 'translation','):', '. .', '..']
X_str = X.astype('str')
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
stoplist = stopwords.words('spanish')
punctuation = set(string.punctuation)
for tweet in X_str:
    tokens=tknzr.tokenize(tweet)
    tokens_ar.append(tokens)
porter = PorterStemmer()
for tokens in tokens_ar:
    # Get lemmas
    lemmas = [porter.stem(t) for t in tokens]
    # Remove unnecessary words
    lemmas_clean = [w for w in lemmas if w not in stoplist]
    # Remove punctuation
    lemmas_punct = [w for w in lemmas_clean if  w not in punctuation]
    # Clean emojis, urls, bars, etc (ATENCIÓN:CREO QUE NO FUNCIONA)
    lemmas_clean = [w for w in lemmas_punct if not w.startswith('@') if w not in pr 
            if not bar.search(w) if not ht.search(w)
            if not w.isdigit()]
    lemmas_ar.append(lemmas_clean)
